# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

R. Burn  ->  R. Burn  |  ['R. Burn']
M. Benisty  ->  M. Benisty  |  ['M. Benisty']
Arxiv has 53 new papers today
          2 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/2 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2502.07294


extracting tarball to tmp_2502.07294...

 done.


R. Burn  ->  R. Burn  |  ['R. Burn']


...

Found 84 bibliographic references in tmp_2502.07294/sample631.bbl.
Issues with the citations
syntax error in line 172: '=' expected
Retrieving document from  https://arxiv.org/e-print/2502.07759


extracting tarball to tmp_2502.07759...

 done.


M. Benisty  ->  M. Benisty  |  ['M. Benisty']


Found 60 bibliographic references in tmp_2502.07759/sample631.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2502.07294-b31b1b.svg)](https://arxiv.org/abs/2502.07294) | **Estimate the Mass Escaping Rates of Radius-valley-spanning Planets in TOI-431 System via XUV Evaporation**  |
|| X. Jiang, J. H. Jiang, <mark>R. Burn</mark>, Z.-H. Zhu |
|*Appeared on*| *2025-02-12*|
|*Comments*| *16 pages, 14 figures, 4 tabels. Accepted by ApJ*|
|**Abstract**|            TOI-431 system has 3 close-in exoplanets, which gives an ideal lab to study gas escape. In this study, we measure the XUV luminosity for TOI-431 with XMM-Newton/EPIC-pn and OM data, then calculate the fluxes for the planets in the system. We find that, TOI-431 b's $\rm F_{XUV,b}=$$70286^{+12060}_{-2611}$$\rm \ erg\ cm^{-2}s^{-1}$ is 75 times of TOI-431 d $\rm F_{XUV,d}=$$935^{+160}_{-35}$$\rm \ erg\ cm^{-2}s^{-1}$. Adopting the energy limit method and hydrodynamic code $ATES$ with a set of He/H ratios, we obtain the mass-loss rates of $10^{10.51^{+0.07}_{-0.02}}$ g s$^{-1}$ for TOI-431 b, $10^{9.14^{+0.07}_{-0.02}}$ and $10^{9.84\sim 9.94}$ g s$^{-1}$ for TOI-431 d. We predict the $2.93\sim 7.91 \%$ H I Ly$\alpha$ and $0.19\sim 10.65\%$ He I triplet absorption depths for TOI-431 d, thus its gas escaping is detectable in principle. For both TOI-431 b and d, we select similar planets from the New Generation Planetary Population Synthesis (NGPPS) data. Then show that considering the mass-loss rates, TOI-431 b should be a naked solid planet, and TOI-431 d will likely maintain its gas envelope until the host star dies. According to the formation and evolution tracks, we find that TOI-431 b's potential birthplace (0.1-2 AU) should be inner than TOI-431 d (2-12 AU). Our results are consistent with the interpretation of the radius valley being caused by atmospheric escape. The intrinsic reason may be their birthplace, which will determine how close they can migrate to the host star, then lose mass and result in the Fulton gap.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2502.07759-b31b1b.svg)](https://arxiv.org/abs/2502.07759) | **A Tentative Detection of a Point Source in the Disk Gap of HD 100546 with VLT/SPHERE-IRDIS Sparse Aperture Masking Interferometry**  |
|| D. Blakely, et al. -- incl., <mark>M. Benisty</mark> |
|*Appeared on*| *2025-02-12*|
|*Comments*| *Accepted for publication in AJ*|
|**Abstract**|            We re-analyze VLT/SPHERE-IRDIS K and H-band sparse aperture masking interferometry data of the transition disk HD 100546 observed in 2018 and 2021, respectively. We fit geometrical models to the closure phases extracted from both datasets. We compare three model classes: a forward scattering disk, a forward scattering disk plus an arbitrary asymmetric disk feature and a forward scattering disk plus an unresolved point source in the disk-gap. We find that the forward scattering disk plus point source model is the best representation of the data. We find that this point source candidate moved from a position of sep. = $39.9^{+2.8}_{-3.3}$ mas, P.A. = $124.1^{+1.0}_{-1.0}$ degrees to a sep. = $50.0^{+1.0}_{-1.0}$ mas, P.A. = $106.4^{+1.4}_{-1.4}$ degrees between 2018 and 2021. Both of these positions are well within the $\sim$13 au ($\sim$120 mas) disk-gap, favouring the point source interpretation. We explore the orbital parameter space that is consistent with the measured relative astrometry. We find orbits either with a similar orientation to the outer disk, with a high eccentricity $e \gtrapprox 0.65$, or orbits with a large relative inclination ($\sim$60 degrees) to the outer disk, and any eccentricity. Despite the significance of the observed point-source signal, follow-up observations will be necessary to conclusively determine its nature.         |

## Failed papers

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2502.07294/./figures/heh_mass_dot_all.png', 'tmp_2502.07294/./figures/density_profiles.png', 'tmp_2502.07294/./figures/transmission_all_ism_2.png']
copying  tmp_2502.07294/./figures/heh_mass_dot_all.png to _build/html/
copying  tmp_2502.07294/./figures/density_profiles.png to _build/html/
copying  tmp_2502.07294/./figures/transmission_all_ism_2.png to _build/html/
exported in  _build/html/2502.07294.md
    + _build/html/tmp_2502.07294/./figures/heh_mass_dot_all.png
    + _build/html/tmp_2502.07294/./figures/density_profiles.png
    + _build/html/tmp_2502.07294/./figures/transmission_all_ism_2.png
found figures ['tmp_2502.07759/./cornerplot.png', 'tmp_2502.07759/./chi2_plot.png', 'tmp_2502.07759/./orbit_plot_comb2.png']
copying  tmp_2502.07759/./cornerplot.png to _build/html/
copying  tmp_2502.07759/./chi2_plot.png to _build/html/
copying  tmp_2502.07759/./orbit_plot_comb2.png to _build/html/
exported in  _build/html/2502.07759.md
    + _build/html/tmp_2502.07759/./c

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$</div>



<div id="title">

# Estimate the Mass Escaping Rates of Radius-valley-spanning Planets in TOI-431 System via XUV Evaporation

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2502.07294-b31b1b.svg)](https://arxiv.org/abs/2502.07294)<mark>Appeared on: 2025-02-12</mark> -  _16 pages, 14 figures, 4 tabels. Accepted by ApJ_

</div>
<div id="authors">

X. Jiang (蒋效铭), J. H. Jiang, <mark>R. Burn</mark>, Z.-H. Zhu

</div>
<div id="abstract">

**Abstract:** TOI-431 system has 3 close-in exoplanets, which gives an ideal lab to study gas escape.In this study, we measure the XUV luminosity for TOI-431 with XMM-Newton/EPIC-pn and OM data, then calculate the fluxes for the planets in the system. We find that, TOI-431 b's $\rm F_{XUV,b}=70286^{+12060}_{-2611}\rm  erg cm^{-2}s^{-1}$ is 75 times of TOI-431 d $\rm F_{XUV,d}=935^{+160}_{-35}\rm  erg cm^{-2}s^{-1}$ .Adopting the energy limit method and hydrodynamic code $ATES$ with a set of He/H ratios, we obtain the mass-loss rates of $10^{10.51^{+0.07}_{-0.02}}$ g s $^{-1}$ for TOI-431 b, $10^{9.14^{+0.07}_{-0.02}}$ and $10^{9.84\sim 9.94}$ g s $^{-1}$ for TOI-431 d. We predict the $2.93\sim 7.91 \%$ H I Ly $\alpha$ and $0.19\sim 10.65\%$ He I triplet absorption depths for TOI-431 d, thus its gas escaping is detectable in principle.For both TOI-431 b and d, we select similar planets from the New Generation Planetary Population Synthesis (NGPPS) data. Then show that considering the mass-loss rates, TOI-431 b should be a naked solid planet, and TOI-431 d will likely maintain its gas envelope until the host star dies.According to the formation and evolution tracks, we find that TOI-431 b's potential birthplace (0.1-2 AU) should be inner than TOI-431 d (2-12 AU).Our results are consistent with the interpretation of the radius valley being caused by atmospheric escape. The intrinsic reason may be their birthplace, which will determine how close they can migrate to the host star, then lose mass and result in the Fulton gap.

</div>

<div id="div_fig1">

<img src="tmp_2502.07294/./figures/heh_mass_dot_all.png" alt="Fig8" width="100%"/>

**Figure 8. -** **Left axis:** relation of He/H ratio and mass loss rate of TOI-431 d, HD205498 b, and GJ436 b from $ATES$(solid lines) and 3D model in \cite{2018MNRAS.481.5315S}(dotted lines); **Right axis:** relation of He/H ratio and average molecular mass of the atmosphere (dashed line). (*fig:heh_mass_dot_all*)

</div>
<div id="div_fig2">

<img src="tmp_2502.07294/./figures/density_profiles.png" alt="Fig9" width="100%"/>

**Figure 9. -** Density profiles of different He/H ratios for GJ 436 b from $ATES$. As the He/H ratio increases, the atmosphere will fall down quickly. (*fig:density_profiles*)

</div>
<div id="div_fig3">

<img src="tmp_2502.07294/./figures/transmission_all_ism_2.png" alt="Fig12" width="100%"/>

**Figure 12. -** Theoretical intrinsic (solid) and ISM absorbed (dashed) transmission profiles of H I Ly$\alpha$(the ISM absorbed depths values are listed after the intrinsic depths) and He I triplet for TOI-431 d with the uncertainties range derived from the mass-loss rate uncertainties. The theoretical intrinsic maximum H I Ly$\alpha$ transit depths are almost unchanged $44.34\%$ at 1215.6 $\rm Å$, but with the ISM absorption, transit depths will decrease continually as He/H ratio goes up, and much weaker than the intrinsic transit depth. The He I triplet absorption depths at 10830.3 $\rm Å$ will increase up to 10.65 $\%$ at He/H of 0.5, then decrease as He/H goes up. (*figures/transmission_all_ism_2*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2502.07294"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\dij}[1]{\textcolor{brown}{#1}}$
$\newcommand{\db}[1]{\textcolor{red}{#1}}$
$\newcommand{\bn}[1]{\textcolor{magenta}{#1}}$</div>



<div id="title">

# A Tentative Detection of a Point Source in the Disk Gap of HD 100546 with VLT/SPHERE-IRDIS Sparse Aperture Masking Interferometry $\footnote{ Based on observations collected at the European Southern Observatory under ESO programmes 2100.C-5052(B), 105.2067.001}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2502.07759-b31b1b.svg)](https://arxiv.org/abs/2502.07759)<mark>Appeared on: 2025-02-12</mark> -  _Accepted for publication in AJ_

</div>
<div id="authors">

D. Blakely, et al. -- incl., <mark>M. Benisty</mark>

</div>
<div id="abstract">

**Abstract:** We re-analyze VLT/SPHERE-IRDIS _K_ and _H_ -band sparse aperture masking interferometry data of the transition disk HD 100546 observed in 2018 and 2021, respectively. We fit geometrical models to the closure phases extracted from both datasets. We compare three model classes: a forward scattering disk, a forward scattering disk plus an arbitrary asymmetric disk feature and a forward scattering disk plus an unresolved point source in the disk-gap. ${We find that the forward scattering disk plus point source model is the best representation of the data.}$ We find that this point source candidate moved from a position of ${sep. = $39.9^{+2.8}_{-3.3}$ mas, P.A. = $124.1^{+1.0}_{-1.0}$ degrees to a sep. = $50.0^{+1.0}_{-1.0}$ mas, P.A. = $106.4^{+1.4}_{-1.4}$}$ degrees between 2018 and 2021. ${Both of these positions are well within the $\sim$13 au ($\sim$120 mas) disk-gap, favouring the point source interpretation}$ . We explore the orbital parameter space that is consistent with the measured relative astrometry. We find orbits either with a similar orientation to the outer disk, with a high eccentricity $e \gtrapprox 0.65$ , or orbits with a large relative inclination ( $\sim$ 60 degrees) to the outer disk, and any eccentricity.Despite the significance of the observed ${point-source}$ signal, follow-up observations will be necessary to conclusively determine its nature.

</div>

<div id="div_fig1">

<img src="tmp_2502.07759/./cornerplot.png" alt="Fig7" width="100%"/>

**Figure 7. -** Corner plot from the joint fit of the PG+PS model, with the disk geometry fixed between epochs, and the companion location allowed to vary. (*fig:corner_plot*)

</div>
<div id="div_fig2">

<img src="tmp_2502.07759/./chi2_plot.png" alt="Fig4" width="100%"/>

**Figure 4. -** {Reduced chi-squared ($\chi^2_{r}$) maps, calculated by fitting a point source (binary) model to the residuals of the PG model fit, at each location (with a step size of 1 mas) within a radius of 125 mas. The significance of the 2018 detection is a $>$8$\sigma$ result and the 2021 detection is a 5$\sigma$ result, using the method by [Gallenne, Mérand and Kervella (2015)](). The 3$\sigma$ credible interval contours on the location of the point source at each epoch are denoted by the teal (2018, left) and maroon (2021, right) contours.} The 90\% and 50\% contours of the disk from the single polar Gaussian (PG) fit are denoted by the black lines. (*fig:chi2_plot*)

</div>
<div id="div_fig3">

<img src="tmp_2502.07759/./orbit_plot_comb2.png" alt="Fig6" width="100%"/>

**Figure 6. -** The white contours show the {median model} disk component of the PG+PS model plotted on top of a $_K_s$_-band total intensity image of HD 100546 from [Ren, Benisty and Ginski (2023)]()(left) and ALMA band 6 data of HD 100546 from [Norfolk, Pinte and Calcino (2022)]()(right). The images are shown on a linear scale. In both panels, the location of the central star is denoted by the white star and the 3 $\sigma$ credible interval on the point source positions from the joint fit to all datasets with the PG+PS model are shown by the teal (2018) and maroon (2021) contours. The beamsize of the ALMA image is shown in the bottom left corner of the right panel. 1000 randomly drawn orbits from the orbital fit posteriors are plotted on top of both images. Note that in the right panel the inner disk in the ALMA image is almost entirely occulted by the plotted orbits. (*fig:joint_fit*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2502.07759"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

331  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

7  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

4  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
